# Web Scraping

### Import Dependencies 

In [1]:
!apt update
!pip install selenium
!pip install webdriver-manager

'apt' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import random
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver

#因为我用的edge浏览器，所以这里用的是edge driver，如果你电脑里装的是chrome，要换成chrome driver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager

In [3]:
info_df = pd.read_csv("../tmdb_5000_data/tmdb_5000_movies.csv")
info_df = info_df.drop_duplicates()
info_df = info_df[['id', 'title','release_date']]
info_df['release_date'] = pd.to_datetime(info_df['release_date'], format = "%Y-%m-%d").dt.strftime('%Y')
info_df.rename(columns= {'id':'movie_id', 'release_date':'startYear'}, inplace = True)
info_df["storyline"] = np.nan
info_df['storyline'] = info_df['storyline'].astype('object')
info_df.head(10)

,movie_id,title,startYear,storyline
0,19995,Avatar,2009,NaN
1,285,Pirates of the Caribbean: At World's End,2007,NaN
2,206647,Spectre,2015,NaN
3,49026,The Dark Knight Rises,2012,NaN
4,49529,John Carter,2012,NaN
5,559,Spider-Man 3,2007,NaN
6,38757,Tangled,2010,NaN
7,99861,Avengers: Age of Ultron,2015,NaN
8,767,Harry Potter and the Half-Blood Prince,2009,NaN
9,209112,Batman v Superman: Dawn of Justice,2016,NaN


In [4]:
imdb_df = pd.read_csv('../tmdb_5000_data/data.tsv', sep = '\t')
imdb_df = imdb_df[imdb_df['titleType'] == 'movie']
imdb_df = imdb_df[['tconst', 'primaryTitle','startYear']]
imdb_df['startYear'] = imdb_df['startYear'].replace('\n','').astype(str)
imdb_df.rename(columns= {'tconst':'imdb_id', 'primaryTitle':'title'}, inplace = True)
imdb_df.head(10)

C:\Users\zjche\AppData\Local\Temp\ipykernel_19760\690390642.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_df = pd.read_csv('../tmdb_5000_data/data.tsv', sep = '\t')


,imdb_id,title,startYear
8,tt0000009,Miss Jerry,1894
144,tt0000147,The Corbett-Fitzsimmons Fight,1897
498,tt0000502,Bohemios,1905
570,tt0000574,The Story of the Kelly Gang,1906
587,tt0000591,The Prodigal Son,1907
610,tt0000615,Robbery Under Arms,1907
625,tt0000630,Hamlet,1908
668,tt0000675,Don Quijote,1908
672,tt0000679,The Fairylogue and Radio-Plays,1908
828,tt0000838,A Cultura do Cacau,1909


In [5]:
full_df = info_df.merge(imdb_df, how = 'inner',on = ['title', 'startYear'])
full_df = full_df.reset_index()
full_df.head()

,index,movie_id,title,startYear,storyline,imdb_id
0,0,19995,Avatar,2009,NaN,tt0499549
1,1,285,Pirates of the Caribbean: At World's End,2007,NaN,tt0449088
2,2,206647,Spectre,2015,NaN,tt2379713
3,3,49026,The Dark Knight Rises,2012,NaN,tt1345836
4,4,49529,John Carter,2012,NaN,tt0401729


### Define url function

In [6]:
def get_url(movie_name):
    url_template = "https://www.imdb.com/title/{}/"
    url = url_template.format(movie_name)
    return url

### Scrape movie reviews from critics

In [7]:
import re

#create driver instance.
driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()))

#iterate through index, reverse() is called so it is easier to see the result and debug. 
indexes = full_df.index.tolist()
indexes.reverse()
movie_num=0
for i in indexes:
    movie_num += 1
    name_og = full_df['title'].iloc[i]
    imdb_id = full_df['imdb_id'].iloc[i]
    
    driver.get(get_url(imdb_id)) #The entire website's html file
    
    #driver.implicitly_wait(0.1)     #Wait time to avoid human verification, 
    storylines = driver.find_elements(By.CLASS_NAME, 'ipc-html-content-inner-div') #Find specific element/elements by class name
    story_collections = []     
    for storyline in storylines:
        result_html = storyline.get_attribute('innerHTML')
        soup = BeautifulSoup(result_html, 'html.parser') #Process the driver-returned elements.
        
        story_collections.append(str(soup).strip())
    
    full_df.at[i, 'storyline'] = story_collections
    print("{}: story added for movie: {}".format(movie_num, name_og))
driver.quit()

1: story added for movie: Shanghai Calling
2: story added for movie: Newlyweds
3: story added for movie: El Mariachi
4: story added for movie: Cavite
5: story added for movie: Primer
6: story added for movie: Bang
7: story added for movie: Cure
8: story added for movie: Tin Can Man
9: story added for movie: The Circle
10: story added for movie: Clean
11: story added for movie: Clean
12: story added for movie: Pink Flamingos
13: story added for movie: All Superheroes Must Die
14: story added for movie: Breaking Upwards
15: story added for movie: Stories of Our Lives
16: story added for movie: The Puffy Chair
17: story added for movie: Flywheel
18: story added for movie: Dry Spell
19: story added for movie: Dutch Kills
20: story added for movie: Slacker
21: story added for movie: Rampage
22: story added for movie: Manito
23: story added for movie: In the Company of Men
24: story added for movie: Funny Ha Ha
25: story added for movie: Pink Narcissus
26: story added for movie: Clerks
27: s

C:\Users\zjche\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


70: story added for movie: Eraserhead
71: story added for movie: The Gallows
72: story added for movie: Facing the Giants
73: story added for movie: The Birth of a Nation
74: story added for movie: A Dog's Breakfast
75: story added for movie: Your Sister's Sister
76: story added for movie: Sound of My Voice
77: story added for movie: An American in Hollywood
78: story added for movie: Roger & Me
79: story added for movie: Top Spin
80: story added for movie: Aroused
81: story added for movie: Antarctic Edge: 70° South
82: story added for movie: Heroes of Dirt
83: story added for movie: When the Lights Went Out
84: story added for movie: The Horse Boy
85: story added for movie: The Woman Chaser
86: story added for movie: Once
87: story added for movie: Tadpole
88: story added for movie: Perfect Cowboy
89: story added for movie: Sweet Sweetback's Baadasssss Song
90: story added for movie: Another Earth
91: story added for movie: The World Is Mine
92: story added for movie: Butterfly Girl


257: story added for movie: The Kentucky Fried Movie
258: story added for movie: Woodstock
259: story added for movie: The Blair Witch Project
260: story added for movie: Top Hat
261: story added for movie: Walter
262: story added for movie: Treachery
263: story added for movie: Treachery
264: story added for movie: After
265: story added for movie: After
266: story added for movie: Starsuckers
267: story added for movie: Escape from Tomorrow
268: story added for movie: Hav Plenty
269: story added for movie: Half Nelson
270: story added for movie: B-Girl
271: story added for movie: L.I.E.
272: story added for movie: Rize
273: story added for movie: Hoop Dreams
274: story added for movie: High Noon
275: story added for movie: Donkey Punch
276: story added for movie: Interview with the Assassin
277: story added for movie: The Conformist
278: story added for movie: The Innkeepers
279: story added for movie: Kill List
280: story added for movie: Kevin Hart: Laugh at My Pain
281: story adde

440: story added for movie: Niagara
441: story added for movie: The Lion of Judah
442: story added for movie: C.H.U.D.
443: story added for movie: The Lost Weekend
444: story added for movie: Friday the 13th Part 2
445: story added for movie: Rebecca
446: story added for movie: Forget Me Not
447: story added for movie: Neal 'n' Nikki
448: story added for movie: The Hebrew Hammer
449: story added for movie: Queen of the Mountains
450: story added for movie: Eddie: The Sleepwalking Cannibal
451: story added for movie: Journey from the Fall
452: story added for movie: Trees Lounge
453: story added for movie: The Celebration
454: story added for movie: Lock, Stock and Two Smoking Barrels
455: story added for movie: Batman
456: story added for movie: House at the End of the Drive
457: story added for movie: The Blue Room
458: story added for movie: The Dog Lover
459: story added for movie: Incident at Loch Ness
460: story added for movie: Shalako
461: story added for movie: The Holy Girl
46

...

3770: story added for movie: Shooter
3771: story added for movie: Alien: Resurrection
3772: story added for movie: London Has Fallen
3773: story added for movie: Joy
3774: story added for movie: Jack Ryan: Shadow Recruit
3775: story added for movie: Under Siege 2: Dark Territory
3776: story added for movie: The Tuxedo
3777: story added for movie: The Tale of Despereaux
3778: story added for movie: Up Close & Personal
3779: story added for movie: Paycheck
3780: story added for movie: The Jackal
3781: story added for movie: ParaNorman
3782: story added for movie: A Civil Action
3783: story added for movie: Hellboy
3784: story added for movie: Jingle All the Way
3785: story added for movie: Starsky & Hutch
3786: story added for movie: The Shaggy Dog
3787: story added for movie: Man on Fire
3788: story added for movie: The Happening
3789: story added for movie: Coraline
3790: story added for movie: Kangaroo Jack
3791: story added for movie: Contagion
3792: story added for movie: This Is It

3957: story added for movie: Funny People
3958: story added for movie: Fool's Gold
3959: story added for movie: The Angry Birds Movie
3960: story added for movie: Sphere
3961: story added for movie: Snake Eyes
3962: story added for movie: Sucker Punch
3963: story added for movie: Minions
3964: story added for movie: Unbreakable
3965: story added for movie: Flight of the Phoenix
3966: story added for movie: Monkeybone
3967: story added for movie: Carriers
3968: story added for movie: Soldier
3969: story added for movie: Soldier
3970: story added for movie: Hollywood Homicide
3971: story added for movie: Titan A.E.
3972: story added for movie: Hostage
3973: story added for movie: Hostage
3974: story added for movie: Immortals
3975: story added for movie: Anna and the King
3976: story added for movie: First Knight
3977: story added for movie: Bandits
3978: story added for movie: Monster House
3979: story added for movie: Spanglish
3980: story added for movie: The Man from U.N.C.L.E.
3981:

4142: story added for movie: Hollow Man
4143: story added for movie: Contact
4144: story added for movie: The Haunted Mansion
4145: story added for movie: A.I. Artificial Intelligence
4146: story added for movie: Pixels
4147: story added for movie: You Don't Mess with the Zohan
4148: story added for movie: Valkyrie
4149: story added for movie: Alvin and the Chipmunks: The Road Chip
4150: story added for movie: Atlantis: The Lost Empire
4151: story added for movie: Ben-Hur
4152: story added for movie: Sherlock Holmes
4153: story added for movie: The Girl with the Dragon Tattoo
4154: story added for movie: Tropic Thunder
4155: story added for movie: Mulan
4156: story added for movie: The Departed
4157: story added for movie: Charlie's Angels
4158: story added for movie: The Secret Life of Walter Mitty
4159: story added for movie: Ice Age: Dawn of the Dinosaurs
4160: story added for movie: Cloudy with a Chance of Meatballs
4161: story added for movie: What Lies Beneath
4162: story added f

4313: story added for movie: Harry Potter and the Prisoner of Azkaban
4314: story added for movie: The Adventures of Tintin
4315: story added for movie: Noah
4316: story added for movie: Salt
4317: story added for movie: Puss in Boots
4318: story added for movie: Bad Boys II
4319: story added for movie: War of the Worlds
4320: story added for movie: Home
4321: story added for movie: Home
4322: story added for movie: The Hunger Games: Catching Fire
4323: story added for movie: Ant-Man
4324: story added for movie: Kung Fu Panda
4325: story added for movie: The Bourne Ultimatum
4326: story added for movie: Penguins of Madagascar
4327: story added for movie: Rango
4328: story added for movie: Turbo
4329: story added for movie: The Revenant
4330: story added for movie: The BFG
4331: story added for movie: The Incredible Hulk
4332: story added for movie: Happy Feet Two
4333: story added for movie: The Twilight Saga: Breaking Dawn - Part 2
4334: story added for movie: Master and Commander: Th

4484: story added for movie: The Hobbit: The Battle of the Five Armies
4485: story added for movie: Men in Black 3
4486: story added for movie: Pirates of the Caribbean: On Stranger Tides
4487: story added for movie: The Avengers
4488: story added for movie: The Chronicles of Narnia: Prince Caspian
4489: story added for movie: Man of Steel
4490: story added for movie: The Lone Ranger
4491: story added for movie: Pirates of the Caribbean: Dead Man's Chest
4492: story added for movie: Quantum of Solace
4493: story added for movie: Superman Returns
4494: story added for movie: Batman v Superman: Dawn of Justice
4495: story added for movie: Harry Potter and the Half-Blood Prince
4496: story added for movie: Avengers: Age of Ultron
4497: story added for movie: Tangled
4498: story added for movie: Spider-Man 3
4499: story added for movie: John Carter
4500: story added for movie: The Dark Knight Rises
4501: story added for movie: Spectre
4502: story added for movie: Pirates of the Caribbean: 

In [8]:
full_df

,index,movie_id,title,startYear,storyline,imdb_id
0,0,19995,Avatar,2009,[The Na'vi language was created entirely from ...,tt0499549
1,1,285,Pirates of the Caribbean: At World's End,2007,"[Take a look back at <a class=""ipc-md-link"" hr...",tt0449088
2,2,206647,Spectre,2015,"[In the <a class=""ipc-md-link ipc-md-link--ent...",tt2379713
3,3,49026,The Dark Knight Rises,2012,"[About a year before this movie's release, wri...",tt1345836
4,4,49529,John Carter,2012,"[While filming at Big Water in Utah, the crew ...",tt0401729
...,...,...,...,...,...,...
4498,4498,14337,Primer,2004,[The budget for the entire film was around $70...,tt0390384
4499,4499,67238,Cavite,2005,"[Adam, a security guard, travels from Californ...",tt0428303
4500,4500,9367,El Mariachi,1992,[El Mariachi just wants to play his guitar and...,tt0104815
4501,4501,72766,Newlyweds,2011,"[At $9000, this is the lowest film budget that...",tt1880418


### Save results

In [9]:
# Convert the dataframe to a csv file
date = datetime.today().strftime('%Y-%m-%d')
full_df.to_csv(date + "movie_with_imdb_storyline.csv", index=False)

In [10]:
# read csv 
full_df = pd.read_csv(date +"movie_with_imdb_storyline.csv")
full_df